In [8]:
import cv2
import os
import numpy as np

In [9]:
import cv2
import numpy as np

def four_point_transform(image, pts):
    (tl, tr, br, bl) = pts[:4]

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([[0, 0],
                    [maxWidth - 1, 0],
                    [maxWidth - 1, maxHeight - 1],
                    [0, maxHeight - 1]], dtype="float32")

    M = cv2.getPerspectiveTransform(pts[:4], dst)
    # or use this function
    # M = cv2.findHomography(pts, dst)[0]
    
    w, h = pts[4:6]
    bl = cv2.perspectiveTransform(np.array([[bl]]), M)[0][0]
    h = cv2.perspectiveTransform(np.array([[h]]), M)[0][0]
    w = cv2.perspectiveTransform(np.array([[w]]), M)[0][0]
    h = np.sqrt(((h[0] - bl[0]) ** 2) + ((h[1] - bl[1]) ** 2))
    w = np.sqrt(((w[0] - bl[0]) ** 2) + ((w[1] - bl[1]) ** 2))
    # convert all image
    warped = cv2.warpPerspective(image, M, image.shape[:2][::-1])

    # return warped
    return M, warped, w, h

# define 4 points for ROI


def selectROI(event, x, y, flags, param):
    global imagetmp, roiPts, ct

    if event == cv2.EVENT_LBUTTONDOWN:
        roiPts[ct] = np.array([x, y], dtype="float32")
        ct += 1
        cv2.circle(imagetmp, (x, y), 2, (0, 255, 0) if ct < 4 else (0, 0, 255), -1)
        # cv2.imshow("image", imagetmp)
        if ct == 4:
            cv2.line(imagetmp, (int(roiPts[0][0]), int(roiPts[0][1])), (int(roiPts[1][0]), int(roiPts[1][1])), (0, 255, 0), 2)
            cv2.line(imagetmp, (int(roiPts[1][0]), int(roiPts[1][1])), (int(roiPts[2][0]), int(roiPts[2][1])), (0, 255, 0), 2)
            cv2.line(imagetmp, (int(roiPts[2][0]), int(roiPts[2][1])), (int(roiPts[3][0]), int(roiPts[3][1])), (0, 255, 0), 2)
            cv2.line(imagetmp, (int(roiPts[3][0]), int(roiPts[3][1])), (int(roiPts[0][0]), int(roiPts[0][1])), (0, 255, 0), 2)

        cv2.imshow("image", imagetmp)


In [10]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython import display

In [11]:
global imagetmp, roiPts, ct
roiPts = np.zeros((6, 2), dtype="float32")
ct = 0

vid = cv2.VideoCapture('1.mp4')
# image = cv2.imread("1.jpg")
image = vid.read()[1]
cv2.imwrite("2.jpg", image)
# imagetmp = cv2.resize(image, (600, 600))
imagetmp = image.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", selectROI)

M = None
warped = None
while ct < 6:
    cv2.imshow("image", imagetmp)
    cv2.waitKey(500)
cv2.destroyAllWindows()

M, warped, w, h = four_point_transform(imagetmp, roiPts)
cv2.imshow("warped", warped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
M

array([[ 1.61315154e+00, -1.34169277e-01, -7.78490707e+02],
       [-3.49815098e-02,  1.78988725e+00, -5.63843635e+01],
       [-7.86822425e-05,  1.26031381e-03,  1.00000000e+00]])

In [13]:
w

86.24348554628749

In [14]:
h

48.474921984338046